In [ ]:
# -*- coding: utf-8 -*-
import cv2
import os
import matplotlib.pyplot as plt
import random
import shutil
from PIL import Image
import numpy as np
import kagglehub

# Download the dataset
dataset_path = kagglehub.dataset_download("kritikseth/fruit-and-vegetable-image-recognition")

if os.path.exists(dataset_path):
    print(f"Dataset downloaded to: {dataset_path}")
    print("Contents of the dataset folder:")
    for root, dirs, files in os.walk(dataset_path):
        print(f"Directory: {root}")
        for file in files:
            print(f"  - {file}")

# Find image files in the dataset path (including subdirectories)
image_files = []
for root, _, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(('.jpg', '.png', '.jpeg')):
            image_files.append(os.path.join(root, file))

# Select a subset of 40–50 images
num_images_to_use = random.randint(40, 50)
selected_images = random.sample(image_files, min(num_images_to_use, len(image_files)))

# Optional: Copy selected images to a new folder for easier management
output_folder = "selected_images"
os.makedirs(output_folder, exist_ok=True)
for img_path in selected_images:
    img_filename = os.path.basename(img_path)
    shutil.copy(img_path, os.path.join(output_folder, img_filename))

# Load and preprocess the selected images
image_data = []
for img_path in selected_images:
    try:
        img = Image.open(img_path).resize((224, 224))  # Resize to 224x224
        img_array = np.array(img) / 255.0  # Normalize pixel values
        image_data.append(img_array)
    except Exception as e:
        print(f"Error processing {img_path}: {e}")

print(f"Loaded {len(image_data)} images for training from {dataset_path}.")

import cv2
import os
import matplotlib.pyplot as plt

# Function to convert image to grayscale and RGB
def process_image(image_path):
    try:
        img = cv2.imread(image_path)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return gray_img, rgb_img
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None, None

# Use selected_images instead of image_data
for img_path in selected_images:  # Iterate through the list of image paths
    gray, rgb = process_image(img_path)

    if gray is not None and rgb is not None:
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(gray, cmap='gray')
        plt.title("Grayscale")
        plt.subplot(1, 2, 2)
        plt.imshow(rgb)
        plt.title("RGB")
        plt.show()

def process_image(image_path):
    try:
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Resize to 200x200
        img_200 = cv2.resize(img_rgb, (200, 200))

        # Resize to 256x256
        img_256 = cv2.resize(img_rgb, (256, 256))

        return img_200, img_256
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None, None

# Iterate directly through selected_images (list of file paths)
for image_path in selected_images:
    img_200, img_256 = process_image(image_path)

    if img_200 is not None and img_256 is not None:
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(img_200)
        plt.title("200x200 RGB")
        plt.subplot(1, 2, 2)
        plt.imshow(img_256)
        plt.title("256x256 RGB")
        plt.show()

# Define dataset split (33% train, 33% val, 33% test)
split_ratio = [0.34, 0.33, 0.33]
train_split = int(split_ratio[0] * len(selected_images))
val_split = int(split_ratio[1] * len(selected_images))

train_images = selected_images[:train_split]
val_images = selected_images[train_split:train_split + val_split]
test_images = selected_images[train_split + val_split:]

# Create output directories
output_dirs = {
    "train": "dataset/train",
    "val": "dataset/val",
    "test": "dataset/test"
}

for _, path in output_dirs.items():
    os.makedirs(path, exist_ok=True)

# Function to copy images to respective directories
def copy_images(image_list, destination_folder):
    for img_path in image_list:
        img_filename = os.path.basename(img_path)
        shutil.copy(img_path, os.path.join(destination_folder, img_filename))

copy_images(train_images, output_dirs["train"])
copy_images(val_images, output_dirs["val"])
copy_images(test_images, output_dirs["test"])

print(f"Dataset spliting completed! \nTrain: {len(train_images)} images\nValidation: {len(val_images)} images\nTest: {len(test_images)} images.")

import os
import shutil

train_dir = "dataset/train"
test_dir = "dataset/test"

# Create class folders (Modify class names based on your dataset)
os.makedirs(os.path.join(train_dir, "class1"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "class2"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "class1"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "class2"), exist_ok=True)

# Move images into class folders (Here, randomly distributing them)
for img in os.listdir(train_dir):
    if img.endswith((".jpg", ".jpeg", ".png")):
        src_path = os.path.join(train_dir, img)
        # Extract the numeric part of the filename using a regular expression
        import re
        numeric_part = re.search(r'\d+', img).group()
        # Use the numeric part for class assignment
        dst_path = os.path.join(train_dir, "class1" if int(numeric_part) % 2 == 0 else "class2", img)
        shutil.move(src_path, dst_path)

for img in os.listdir(test_dir):
    if img.endswith((".jpg", ".jpeg", ".png")):
        src_path = os.path.join(test_dir, img)
        # Extract the numeric part of the filename using a regular expression
        import re
        numeric_part = re.search(r'\d+', img).group()
        # Use the numeric part for class assignment
        dst_path = os.path.join(test_dir, "class1" if int(numeric_part) % 2 == 0 else "class2", img)
        shutil.move(src_path, dst_path)

print("Dataset structure fixed!")

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set parameters
IMG_SIZE = (200, 200)  # Change to (256,256) if needed
COLOR_MODE = "rgb"  # Change to "grayscale" if needed
INPUT_SHAPE = (200, 200, 3) if COLOR_MODE == "rgb" else (200, 200, 1)
BATCH_SIZE = 32
EPOCHS = 10

# Data augmentation and loading
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    "dataset/train",  # Verify this path is correct
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    color_mode=COLOR_MODE,
    subset="training"
)

val_generator = datagen.flow_from_directory(
    "dataset/train",  # Verify this path is correct
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    color_mode=COLOR_MODE,
    subset="validation"
)

test_generator = datagen.flow_from_directory(
    "dataset/test",  # Verify this path is correct
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    color_mode=COLOR_MODE,
    shuffle=False
)

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=INPUT_SHAPE),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation="softmax")
])

# Compile model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train model
history = model.fit(train_generator, validation_data=val_generator, epochs=EPOCHS)

# Evaluate model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# Save model
model.save("car_crash_cnn_model.h5")

# Import the scipy.stats module
from scipy import stats

# Set parameters
IMG_SIZE = (200, 200)  # Change to (256,256) if needed
COLOR_MODE = "rgb"  # Change to "grayscale" if needed
INPUT_SHAPE = (200, 200, 3) if COLOR_MODE == "rgb" else (200, 200, 1)
BATCH_SIZE = 32
EPOCHS = 10

# ... (rest of your code) ...

train_accuracies = history.history['accuracy']
val_accuracies = history.history['val_accuracy']

train_losses = history.history['loss']
val_losses = history.history['val_loss']

train_accuracies = np.array(train_accuracies)
val_accuracies = np.array(val_accuracies)

# Perform Z-test (using ttest_ind)
z_score, p_value_z = stats.ttest_ind(train_accuracies, val_accuracies, equal_var=True)
print(f"\nZ-test Results:\nZ-score = {z_score}\nP-value = {p_value_z}")

# Perform T-test (assuming small sample size or unknown variances)
t_score, p_value_t = stats.ttest_ind(train_accuracies, val_accuracies, equal_var=False)
print(f"\nT-test Results:\nT-score = {t_score}\nP-value = {p_value_t}")

from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

test_loss, test_acc = model.evaluate(test_generator)

# Predict on test data
y_pred_probs = model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)

# True labels
y_true = test_generator.classes

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

# Optional: Classification report
print("\nClassification Report:")
target_names = list(test_generator.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=target_names))

# Optional: Visualize confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names,
            yticklabels=target_names)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()
